# Front-End do Receptor Coerente

Neste notebook iremos explorar um receptor óptico digital coerente típico. O front-end óptico é responsável por produzir uma saída que corresponde as componentes em fase e quadratura de duas orientações de polarização ortogonais que são representadas por correntes elétricas.

Essas correntes são convertidas em tensão e digitalizados por um conjunto de ADCs, gerando os sinais digitais que são enviados para um segundo subsistema, composto por uma cadeia de algoritmos de processamento digital de sinais (DSP) que tem por objetivo realizar as devidas compensações impostas pelo canal. 

Iremos apresentar o front-end do receptor e discutir os algoritmos usados para corrigir suas imperfeições usando métodos de enquadramento e ortogonalização.

### 4.1 The Receiver Front-end

Nos receptores ópticos, a conversão do sinal óptico em correntes elétricas é realizada pelo fotodetector. Fotodetectores são dispositivos optoeletrônicos que geram uma corrente elétrica $I_p(t)$ que é proporcional ao módulo quadrático do campo óptico de entrada $E_{in}(t)$

$$I_{p}(t)=R|E_{i n}(t)|^{2}$$

onde $R$ é uma constante de proporcionalidade chamada responsividade do fotodetector, medida em unidades de $A/W$. Em sistemas ópticos com modulação de intensidade e detecção direta, a conversão optoelétrica é realizada por um único fotodetector. Porém, um fotodetector não é capaz de recuperar a fase do campo de entrada.  Alternativamente, os receptores coerentes recorrem a estruturas mais elaboradas para recuperar a informação transmitida na amplitude, fase e polarização do sinal óptico.

Por definição receptores coerentes são estruturas que misturam um sinal de entrada na frequência $\omega_c$, com um oscilador local na frequência $\omega_{LO}$, para recuperar a fase da portadora de entrada. A mistura é uma operação não linear que converte o sinal na frequência $\omega_c$ para uma frequência intermediaria que denotamos por $\Delta\omega=\omega_{c}-\omega_{LO}$. A detecção coerente pode ser classificada em homódina, heteródina e intradina.

#### Tipos de detecção coerente

- **Detecção Homódina** ($\omega_{c}\;=\;\omega_{LO}$): a frequência do laser do transmissor e a frequência do laser oscilador local são idênticas.

- **Detecção Heteródina** ($\omega_{c}\neq\omega_{LO}$): a frequência do laser do transmissor e a frequência do laser oscilador local são diferentes.

- **Detecção intradina** ($\omega_{c}\approx\omega_{LO}$): a frequência do laser do transmissor e do laser oscilador local são "ligeiramente" diferentes.

Nos casos em que temos uma detecção intradina a conversão para banda base é realizada por algoritmos de recuperação de frequência implementados pelo DSP.

O front-end óptico de um receptor coerente é a estrutura que mistura o sinal óptico de entrada com o laser oscilador local, gerando correntes elétricas, esse dispositivo e denominado como *Hybrid*. Abaixo podemos observar um circuito fotônico que implementa uma híbrida.

<br>
<center>
    <img src="https://i.postimg.cc/rFFBm1Vj/Hybrid-90.png">
</center>
<br>

**Referência:** Digital Coherent Optical Systems: Architecture and Algorithms